In [3]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import csv

class ShareholdingExtractor:
    def __init__(self):
        """Initialize the shareholding pattern extractor"""
        self.setup_driver()
        self.base_url = "https://www.screener.in/company/{}/#shareholding"
        
    def setup_driver(self):
        """Setup Chrome driver with optimized options"""
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Remove this line if you want to see the browser
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 15)
        
    def read_symbols(self, file_path):
        """Read symbols from CSV file"""
        try:
            df = pd.read_csv(file_path)
            if 'SYMBOL' in df.columns:
                return df['SYMBOL'].tolist()
            else:
                print("❌ 'SYMBOL' column not found in CSV")
                return []
        except Exception as e:
            print(f"❌ Error reading symbols: {e}")
            return []
    
    def click_button_and_wait(self, button_text, timeout=10):
        """Click quarterly or yearly button and wait for data to load"""
        try:
            # Find and click the button
            button_xpath = f"//button[contains(text(), '{button_text}')]"
            button = self.wait.until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
            button.click()
            
            # Wait a moment for the data to load
            time.sleep(3)
            print(f"✅ Clicked {button_text} button")
            return True
            
        except TimeoutException:
            print(f"❌ Could not find or click {button_text} button")
            return False
        except Exception as e:
            print(f"❌ Error clicking {button_text} button: {e}")
            return False
    
    def extract_shareholding_data(self, data_type="Quarterly"):
        """Extract shareholding pattern data from the current page"""
        try:
            shareholding_data = []
            
            # Wait for the page to load
            time.sleep(3)
            
            # Look specifically for the shareholding pattern section
            # Try to find the section with "Shareholding Pattern" heading
            try:
                # Look for the specific shareholding table under the "Shareholding Pattern" section
                shareholding_section = self.driver.find_element(By.XPATH, "//h2[contains(text(), 'Shareholding Pattern')]")
                print(f"✅ Found Shareholding Pattern section")
                
                # Find the table that follows this heading
                parent_section = shareholding_section.find_element(By.XPATH, "./following-sibling::*")
                table = parent_section.find_element(By.TAG_NAME, "table")
                
            except:
                # Alternative: Look for table with shareholding categories
                print("🔍 Searching for table with Promoters/FIIs...")
                tables = self.driver.find_elements(By.TAG_NAME, "table")
                table = None
                
                for t in tables:
                    table_text = t.text.lower()
                    # Check if this table contains shareholding pattern keywords
                    if any(keyword in table_text for keyword in ['promoters', 'fiis', 'diis', 'government', 'public']):
                        table = t
                        print(f"✅ Found shareholding table with keywords")
                        break
                
                if not table:
                    print("❌ Could not find shareholding pattern table")
                    return []
            
            # Extract data from the found table
            rows = table.find_elements(By.TAG_NAME, "tr")
            if len(rows) < 2:
                print("❌ Table doesn't have enough rows")
                return []
            
            # Get header row (time periods)
            header_row = rows[0]
            time_periods = []
            header_cells = header_row.find_elements(By.TAG_NAME, "th")
            
            for cell in header_cells[1:]:  # Skip first column (category names)
                period_text = cell.text.strip()
                if period_text:
                    time_periods.append(period_text)
            
            print(f"📅 Found time periods: {time_periods}")
            
            # Extract data rows
            for row in rows[1:]:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) < 2:
                    continue
                
                category = cells[0].text.strip()
                
                # Skip empty categories
                if not category:
                    continue
                
                # Handle different types of data
                values = [cell.text.strip() for cell in cells[1:]]
                
                for i, period in enumerate(time_periods):
                    if i < len(values) and values[i]:
                        value = values[i].strip()
                        
                        # Skip empty or dash values
                        if not value or value == '-' or value == '':
                            continue
                        
                        # Determine unit and clean value
                        unit = 'Count' if 'No. of Shareholders' in category else 'Percentage'
                        
                        # Clean percentage values
                        if unit == 'Percentage':
                            clean_value = value.replace('%', '').replace(',', '').strip()
                        else:
                            clean_value = value.replace(',', '').strip()
                        
                        # Validate that we have a numeric value
                        try:
                            float(clean_value)
                            shareholding_data.append({
                                'Data_Type': data_type,
                                'Category': category,
                                'Period': period,
                                'Value': clean_value,
                                'Unit': unit
                            })
                        except ValueError:
                            # Skip non-numeric values
                            continue
            
            print(f"✅ Extracted {len(shareholding_data)} shareholding data points for {data_type}")
            
            # Debug: Print first few extracted items
            if shareholding_data:
                print("📋 Sample extracted data:")
                for item in shareholding_data[:3]:
                    print(f"   {item['Category']}: {item['Value']} ({item['Period']})")
            
            return shareholding_data
            
        except Exception as e:
            print(f"❌ Error extracting shareholding data: {e}")
            return []
    
    def process_symbol(self, symbol):
        """Process a single symbol and extract both quarterly and yearly data"""
        try:
            print(f"\n🔄 Processing {symbol}...")
            url = self.base_url.format(symbol)
            
            self.driver.get(url)
            time.sleep(5)  # Wait for initial page load
            
            all_data = []
            
            # Extract Quarterly Data
            print("📊 Extracting Quarterly data...")
            if self.click_button_and_wait("Quarterly"):
                quarterly_data = self.extract_shareholding_data("Quarterly")
                for item in quarterly_data:
                    item['Symbol'] = symbol
                all_data.extend(quarterly_data)
            
            # Wait before switching to yearly
            time.sleep(2)
            
            # Extract Yearly Data  
            print("📊 Extracting Yearly data...")
            if self.click_button_and_wait("Yearly"):
                yearly_data = self.extract_shareholding_data("Yearly")
                for item in yearly_data:
                    item['Symbol'] = symbol
                all_data.extend(yearly_data)
            
            return all_data
            
        except Exception as e:
            print(f"❌ Error processing {symbol}: {e}")
            return []
    
    def save_to_csv(self, all_data, output_dir):
        """Save extracted data to CSV in readable pivot format"""
        try:
            if not all_data:
                print("❌ No data to save")
                return
            
            # Create output directory
            os.makedirs(output_dir, exist_ok=True)
            
            # Convert to DataFrame
            df = pd.DataFrame(all_data)
            
            # Group by Symbol and Data_Type for separate processing
            symbols = df['Symbol'].unique()
            
            for symbol in symbols:
                symbol_data = df[df['Symbol'] == symbol]
                
                # Process Quarterly data
                quarterly_data = symbol_data[symbol_data['Data_Type'] == 'Quarterly']
                if not quarterly_data.empty:
                    quarterly_pivot = self.create_readable_format(quarterly_data, symbol, 'Quarterly')
                    quarterly_file = os.path.join(output_dir, f'{symbol}_Quarterly_Shareholding.csv')
                    quarterly_pivot.to_csv(quarterly_file, index=False)
                    print(f"✅ Quarterly data saved: {quarterly_file}")
                
                # Process Yearly data
                yearly_data = symbol_data[symbol_data['Data_Type'] == 'Yearly']
                if not yearly_data.empty:
                    yearly_pivot = self.create_readable_format(yearly_data, symbol, 'Yearly')
                    yearly_file = os.path.join(output_dir, f'{symbol}_Yearly_Shareholding.csv')
                    yearly_pivot.to_csv(yearly_file, index=False)
                    print(f"✅ Yearly data saved: {yearly_file}")
            
            print(f"\n📊 Summary:")
            print(f"📈 Symbols processed: {len(symbols)}")
            print(f"📋 Files created: {len(symbols) * 2} (Quarterly + Yearly for each symbol)")
            
        except Exception as e:
            print(f"❌ Error saving data: {e}")
    
    def create_readable_format(self, data, symbol, data_type):
        """Create readable pivot table format like screener.in"""
        try:
            # Pivot the data: Categories as rows, Periods as columns
            pivot_df = data.pivot_table(
                index='Category', 
                columns='Period', 
                values='Value', 
                aggfunc='first',
                fill_value=''
            )
            
            # Reset index to make Category a regular column
            pivot_df = pivot_df.reset_index()
            
            # Get period columns and sort them chronologically
            period_columns = [col for col in pivot_df.columns if col != 'Category']
            period_columns = self.sort_periods_chronologically(period_columns)
            
            # Reorder columns: Category first, then chronologically sorted periods
            column_order = ['Category'] + period_columns
            pivot_df = pivot_df[column_order]
            
            # Format values with % sign for percentage categories
            for col in period_columns:
                pivot_df[col] = pivot_df.apply(
                    lambda row: f"{row[col]}%" if row[col] != '' and self.is_percentage_category(row['Category']) else row[col], 
                    axis=1
                )
            
            # Clean up category names (remove + signs if present)
            pivot_df['Category'] = pivot_df['Category'].str.replace(' +', '', regex=False)
            
            # Sort categories in logical order
            category_order = ['Promoters', 'FIIs', 'DIIs', 'Government', 'Public', 'No. of Shareholders']
            pivot_df['sort_order'] = pivot_df['Category'].apply(
                lambda x: category_order.index(x) if x in category_order else len(category_order)
            )
            pivot_df = pivot_df.sort_values('sort_order').drop('sort_order', axis=1)
            
            return pivot_df
            
        except Exception as e:
            print(f"❌ Error creating readable format: {e}")
            return pd.DataFrame()
    
    def sort_periods_chronologically(self, periods):
        """Sort periods in chronological order"""
        try:
            # Create a list to store periods with their sort keys
            period_data = []
            
            for period in periods:
                sort_key = self.get_period_sort_key(period)
                period_data.append((period, sort_key))
            
            # Sort by the sort key
            period_data.sort(key=lambda x: x[1])
            
            # Return just the period names in sorted order
            return [period for period, _ in period_data]
            
        except Exception as e:
            print(f"❌ Error sorting periods: {e}")
            return sorted(periods)  # Fallback to alphabetical sort
    
    def get_period_sort_key(self, period):
        """Generate a sort key for chronological ordering"""
        try:
            # Handle different period formats
            period = period.strip()
            
            # For quarterly data (e.g., "Mar-23", "Jun-24")
            if '-' in period and len(period.split('-')) == 2:
                month_str, year_str = period.split('-')
                
                # Map month abbreviations to numbers
                month_map = {
                    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
                }
                
                month_num = month_map.get(month_str, 0)
                
                # Convert 2-digit year to 4-digit (assuming 2000s)
                if len(year_str) == 2:
                    year_num = 2000 + int(year_str)
                else:
                    year_num = int(year_str)
                
                # Return sort key: year * 100 + month (e.g., 202203 for Mar-22)
                return year_num * 100 + month_num
            
            # For yearly data (e.g., "2023", "2024")
            elif period.isdigit():
                return int(period) * 100  # Multiply by 100 to align with quarterly format
            
            # For full month names with year (e.g., "March 2023")
            elif ' ' in period:
                parts = period.split(' ')
                if len(parts) == 2:
                    month_str, year_str = parts
                    month_map = {
                        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12,
                        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
                    }
                    month_num = month_map.get(month_str, 0)
                    year_num = int(year_str)
                    return year_num * 100 + month_num
            
            # Fallback: return a default sort key
            return 999999
            
        except Exception as e:
            print(f"❌ Error processing period '{period}': {e}")
            return 999999  # Put problematic periods at the end
    
    def is_percentage_category(self, category):
        """Check if a category should have percentage formatting"""
        percentage_categories = ['Promoters', 'FIIs', 'DIIs', 'Government', 'Public']
        # Clean category name for comparison
        clean_category = category.replace(' +', '').strip()
        return clean_category in percentage_categories
    
    def run_extraction(self, symbol_file_path, output_dir):
        """Main method to run the extraction process"""
        try:
            print("🚀 Starting CLASS-6: Shareholding Pattern Extraction")
            print("=" * 60)
            
            # Read symbols
            symbols = self.read_symbols(symbol_file_path)
            if not symbols:
                return
            
            print(f"📁 Found {len(symbols)} symbols to process")
            
            all_extracted_data = []
            
            # Process each symbol
            for i, symbol in enumerate(symbols, 1):
                print(f"\n[{i}/{len(symbols)}] Processing: {symbol}")
                symbol_data = self.process_symbol(symbol)
                all_extracted_data.extend(symbol_data)
                
                # Small delay between symbols
                time.sleep(2)
            
            # Save all data
            self.save_to_csv(all_extracted_data, output_dir)
            
            print(f"\n✅ CLASS-6 Complete!")
            print(f"📊 Successfully extracted shareholding data for {len(symbols)} symbols")
            
        except Exception as e:
            print(f"❌ Error in extraction process: {e}")
        finally:
            self.driver.quit()

# Usage Example
if __name__ == "__main__":
    # Configuration
    SYMBOL_FILE = r"E:/JN/TestSymbol.csv"  # Your symbol file path
    OUTPUT_DIR = r"E:/JN/Shareholding_Output/"  # Output directory
    
    # Create extractor and run
    extractor = ShareholdingExtractor()
    extractor.run_extraction(SYMBOL_FILE, OUTPUT_DIR)

🚀 Starting CLASS-6: Shareholding Pattern Extraction
📁 Found 1 symbols to process

[1/1] Processing: RELIANCE

🔄 Processing RELIANCE...
📊 Extracting Quarterly data...
✅ Clicked Quarterly button
✅ Found Shareholding Pattern section
🔍 Searching for table with Promoters/FIIs...
✅ Found shareholding table with keywords
📅 Found time periods: ['Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023', 'Sep 2023', 'Dec 2023', 'Mar 2024', 'Jun 2024', 'Sep 2024', 'Dec 2024', 'Mar 2025', 'Jun 2025']
✅ Extracted 72 shareholding data points for Quarterly
📋 Sample extracted data:
   Promoters +: 50.56 (Sep 2022)
   Promoters +: 50.49 (Dec 2022)
   Promoters +: 50.41 (Mar 2023)
📊 Extracting Yearly data...
✅ Clicked Yearly button
✅ Found Shareholding Pattern section
🔍 Searching for table with Promoters/FIIs...
✅ Found shareholding table with keywords
📅 Found time periods: ['Mar 2017', 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Mar 2024', 'Mar 2025', 'Jun 2025']
✅ Extracted 60 shareh